In [9]:
import os
import sqlite3

import dask.dataframe as dd
import pandas as pd

from scripts.common.helper.entities import DatasetFrame
from scripts.common.helper.entities import IntrinsicsFrameResult, DatasetEntity
from scripts.common.load_env import load_env
from scripts.local.paper.helper.utils import df_from_sql_table, compute_metrics, \
    ddf_from_sqlite_table, metrics_from_labels

load_env()
DB_PATH = os.getenv("LOCAL_SQLITE_MASTER_DB")
robust_point_count_threshold = 64
experiment_id = 38

In [2]:
conn = sqlite3.connect(DB_PATH)
datasets_df = ddf_from_sqlite_table(DB_PATH, DatasetEntity.__table__)
dataset_frames_df = ddf_from_sqlite_table(DB_PATH, DatasetFrame.__table__)
frame_results_df = ddf_from_sqlite_table(DB_PATH, IntrinsicsFrameResult.__table__)
frame_results_df = frame_results_df[frame_results_df["experiment_id"] == experiment_id]
frame_gt_df = ddf_from_sqlite_table(DB_PATH, "dataset_frame_empirical")
scanline_gt_df = ddf_from_sqlite_table(DB_PATH, "dataset_frame_scanline_info_empirical")
scanline_gt_df = df_from_sql_table(conn, "dataset_frame_scanline_info_empirical")
non_robust_frame_ids = scanline_gt_df[scanline_gt_df["points_count"] < robust_point_count_threshold]["dataset_frame_id"].unique()
scanline_gt_df = ddf_from_sqlite_table(DB_PATH, "dataset_frame_scanline_info_empirical")
conn.close()

In [27]:
frame_results_df = frame_results_df.assign(frame_result_id=frame_results_df.index)
frame_full_df = dd.merge(dataset_frames_df, frame_results_df, left_on="id", right_on="dataset_frame_id", how="inner", suffixes=("_frame", "_frame_result"))
frame_full_df = dd.merge(datasets_df, frame_full_df, left_on="id", right_on="dataset_id", how="inner")
frame_full_df = dd.merge(frame_full_df, frame_gt_df, on="dataset_frame_id", how="inner", suffixes=("_estimated", "_gt"))
frame_full_df["robust"] = ~frame_full_df["dataset_frame_id"].isin(non_robust_frame_ids)

In [4]:
def make_metrics_agg(true_col, pred_col):
    def compute_group_metrics(df):
        return compute_metrics(df, true_col, pred_col)
    return compute_group_metrics

In [5]:
metrics_agg = make_metrics_agg("scanlines_count_gt", "scanlines_count_estimated")

meta = pd.DataFrame({
    'samples': pd.Series(dtype='int64'),
    'incorrect': pd.Series(dtype='int64'),
    'oa': pd.Series(dtype='float64'),
    'mp': pd.Series(dtype='float64'),
    'mr': pd.Series(dtype='float64'),
    'mf1': pd.Series(dtype='float64'),
    'wp': pd.Series(dtype='float64'),
    'wr': pd.Series(dtype='float64'),
    'wf1': pd.Series(dtype='float64')
})

scanline_metrics_all_df = frame_full_df.groupby(["name"]) \
    .apply(metrics_agg, meta=meta, include_groups=False) \
    .reset_index() \
    .assign(subset="All")

scanline_metrics_robust_only_df = frame_full_df[frame_full_df["robust"]].groupby(["name"]) \
    .apply(metrics_agg, meta=meta, include_groups=False) \
    .reset_index() \
    .assign(subset="n(l) >= 64")

scanline_metrics_all_df = scanline_metrics_all_df.clear_divisions()
scanline_metrics_robust_only_df = scanline_metrics_robust_only_df.clear_divisions()

scanline_metrics_df = dd.concat([scanline_metrics_all_df, scanline_metrics_robust_only_df], axis=0)
scanline_metrics_df = scanline_metrics_df.clear_divisions()
scanline_metrics_df = scanline_metrics_df.sort_values(by=["name", "subset"], ascending=[False, True])

In [6]:
scanline_metrics_df.compute()

,name,samples,incorrect,oa,mp,mr,mf1,wp,wr,wf1,subset
1,kitti,47885.0,0.0,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,All
1,kitti,47543.0,0.0,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,n(l) >= 64
0,durlar,145916.0,130.0,99.910908,92.977032,93.668454,93.307977,99.920694,99.910908,99.915145,All
0,durlar,130757.0,0.0,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,n(l) >= 64


# Resolutions

In [14]:
def compute_reduced_metrics(df, true_col, pred_col):
    y_true = df[true_col].to_numpy()
    y_pred = df[pred_col].to_numpy()

    to_keep = ["samples", "incorrect", "oa"]
    all_metrics = metrics_from_labels(y_true, y_pred, to_keep)
    reduced_metrics = {k: all_metrics[k] for k in to_keep}

    return pd.Series(reduced_metrics)

def make_reduced_metrics_agg(true_col, pred_col):
    def compute_group_metrics(df):
        return compute_reduced_metrics(df, true_col, pred_col)
    return compute_group_metrics

In [30]:
scanline_results_df = ddf_from_sqlite_table(DB_PATH, "intrinsics_result_scanline_info")
scanlines_full_df = dd.merge(frame_full_df, scanline_results_df, left_on="frame_result_id", right_on="intrinsics_result_id", how="inner", suffixes=("_frame_result", "_scanline_result"))
scanlines_full_df = dd.merge(scanlines_full_df, scanline_gt_df, on=["dataset_frame_id", "scanline_idx"], how="inner", suffixes=("", "_gt"))

scanlines_full_df = scanlines_full_df[scanlines_full_df["experiment_id"] == experiment_id]

In [32]:
reduced_meta = meta = pd.DataFrame({
    'samples': pd.Series(dtype='int64'),
    'incorrect': pd.Series(dtype='int64'),
    'oa': pd.Series(dtype='float64'),
})
reduced_metrics_agg = make_reduced_metrics_agg("horizontal_resolution_gt", "horizontal_resolution")

resolution_metrics_all_df = scanlines_full_df.groupby(["name"]) \
    .apply(reduced_metrics_agg, meta=reduced_meta, include_groups=False) \
    .reset_index() \
    .assign(subset="All")

In [33]:
resolution_metrics_all_df.compute()

KeyboardInterrupt: 

In [31]:
print(scanlines_full_df.columns)

Index(['name', 'laser_count', 'dataset_id', 'relative_path', 'experiment_id',
       'dataset_frame_id', 'points_count_estimated',
       'scanlines_count_estimated', 'vertical_iterations', 'unassigned_points',
       'end_reason', 'frame_result_id', 'points_count_gt',
       'scanlines_count_gt', 'robust', 'intrinsics_result_id', 'scanline_idx',
       'points_count', 'vertical_offset', 'vertical_angle',
       'vertical_ci_offset_lower', 'vertical_ci_offset_upper',
       'vertical_ci_angle_lower', 'vertical_ci_angle_upper',
       'vertical_theoretical_angle_bottom_lower',
       'vertical_theoretical_angle_bottom_upper',
       'vertical_theoretical_angle_top_lower',
       'vertical_theoretical_angle_top_upper', 'vertical_uncertainty',
       'vertical_last_scanline', 'vertical_hough_votes', 'vertical_hough_hash',
       'horizontal_offset', 'horizontal_resolution', 'horizontal_heuristic',
       'horizontal_angle_offset', 'laser_idx', 'points_count_gt',
       'vertical_offset_gt